# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Frontend
💩

# Coordle Backend
Essentially need a good way to do kNN search. 


In [1]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy

import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
df.columns

Index(['cord_uid', 'paper_id', 'source', 'is_pmc', 'title', 'body_text', 'doi',
       'pubmed_id', 'license', 'abstract', 'publish_time', 'authors',
       'journal', 'url', 'language'],
      dtype='object')

In [4]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, logs_filename: str):
        self.output_dir = output_dir
        self.prefix = prefix
        self.logs_filename = logs_filename
        self.epoch = 0

    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        if self.epoch == 0:
            loss = cum_loss
        else:
            loss = cum_loss - self.loss_previous_step
        self.loss_previous_step = loss
        with open(join_path(self.output_dir, self.logs_filename), 'a+') as file:
            file.write(f'Epoch #{self.epoch}, loss: {loss}\n')
        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

In [5]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [6]:
W = model.trainables.syn1neg

In [25]:
# %%time
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index

docsample = df.iloc[0]
def test_SentVectorDoc():
    doc = SentVectorDoc(W, int_to_word, word_to_int, docsample['cord_uid'], 
                        store_sents=True)
    doc, _ = doc.fit(docsample['body_text'])
    print(doc.tf_idf_score)
    return doc

def test_Index():
    coordle = Index(W, int_to_word, word_to_int)
    for i in tqdm(range(128), position=0):
        sample = df.iloc[i]
        coordle.add(sample['cord_uid'], sample['title'], sample['body_text'])
    return coordle

index = test_Index()
# fuck = test_SentVectorDoc()

100%|██████████| 128/128 [00:01<00:00, 81.62it/s]


In [26]:
results = index['retarded white woman']
# [print(result._tf_idf_score) for result in results]
for res, score in zip(*results):
    print(res.uid, score)

Query tokens:  ['retarded', 'white', 'woman']
zp9k1k3z 0.003937323529173992
5xswwney 0.0025311665707896253
fdtwagr1 0.0022391159142015546
hnl4a33n 0.0020737272359471226
1rzbaj02 0.0019657469998755617
2ole6ya7 0.0015826874925502022
vs5yondw 0.0014266594616439065
eyk015n3 0.0012717443626019324
bgygebgx 0.0011908829236034638
ak7bf0gq 0.0011336476320710932
y2ruobm0 0.0010934790151866844
uq88w4fm 0.0010765838077918756
mnhg329v 0.0010675742366997407
i94lyfsh 0.0009631800515236531
y2uhnlpd 0.000916057912493545
ensep6lk 0.0007936163278726123
9jpc42h1 0.0007421340543952382
34jn37sb 0.0007126962553809504
rwlryr0h 0.0006910479034395635
vdk2pxkk 0.0006331542625928979
ootj52fs 0.0006175226343580554
67kyfybe 0.0005973089192080529
9hru9fyg 0.0005376984816147478
uhrijlyz 0.0004899197469188614
qg2pb884 0.0004329878581733049
pq0atbfg 0.0003680755426050817
x8uzlsn7 0.0002594724825448824
qwh8ei60 0.00021505510635494993
pbhdzshl 0.0002081321408991797
h1gnp62r 0.0001863078951119297
8rs8ilio 0.00016480858617